## Without web search

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano"
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

In [4]:
print(response['messages'][-1].content)

Short answer: my knowledge is not real-time. For most GPT-4 based deployments, my training data goes up to around 2023-09. I don’t automatically browse the web unless a browsing tool is enabled in this session.

What this means:
- I can answer most questions about concepts, history, and topics up to that point.
- I may miss or be unsure about events, releases, or developments after that date.
- If you need current information, you can enable a browser tool (if available here) or share updated sources, and I can help summarize or reason about them.


In [5]:
from pprint import pprint
pprint(response)

{'messages': [HumanMessage(content='How up to date is your training knowledge?', additional_kwargs={}, response_metadata={}, id='5548e21e-466d-446f-a1e2-3fe133bdcc86'),
              AIMessage(content='Short answer: my knowledge is not real-time. For most GPT-4 based deployments, my training data goes up to around 2023-09. I don’t automatically browse the web unless a browsing tool is enabled in this session.\n\nWhat this means:\n- I can answer most questions about concepts, history, and topics up to that point.\n- I may miss or be unsure about events, releases, or developments after that date.\n- If you need current information, you can enable a browser tool (if available here) or share updated sources, and I can help summarize or reason about them.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1410, 'prompt_tokens': 15, 'total_tokens': 1425, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_to

In [6]:
# Example to compare the answer Agent answer with chat model
from langchain.chat_models import init_chat_model

chat_model = init_chat_model(
    model="gpt-5-nano")

response = chat_model.invoke(["How up to date is your training knowledge"])
                             
print(response.content)

- I’m trained on data up to about June 2024. I don’t have direct knowledge of events or developments that happened after that unless you share new information or I’m explicitly given access to a browsing tool.

What that means:
- I can help with general concepts, reasoning, and information up to mid-2024.
- I can’t automatically fetch or verify the latest news, edge-case updates, or newly released data beyond my cutoff unless browsing or an external source is used.
- If you need the most current details (e.g., latest tech releases, policy changes), you can tell me to search the web if browsing is available, or provide the latest facts and I’ll help interpret them.


## My question to CoPilot
Why does the agentic answer in cell 3 and 4 say that the model was trained with info up to late 2023 and the chat model answer in later cells says June 2024?

Note: I got different answers in other runs ...

## Answer
The discrepancy you're seeing is because the two models (used by the agent and the chat model) have different knowledge cutoffs based on when their training data ends. This is common with different model versions or configurations:

The agentic answer (from the model used in the agent setup) reports a cutoff of "late 2023" (with some updates into early 2024).
The chat model answer reports a cutoff of "June 2024".
Even though the notebook code specifies "gpt-5-nano" for both, in practice, this could resolve to different underlying models or versions depending on your environment/configuration (e.g., one might be an older GPT-4 variant, while the other is a newer GPT-4o or similar). Model providers often release updates with extended cutoffs, and different instances or API calls might use different versions.

If this is intentional for the demo, it highlights how agents can use tools to access real-time data beyond their base training cutoff, while direct model queries are limited to their trained knowledge. If you want consistent cutoffs, ensure both use the exact same model version in your setup.

In [7]:
# Underlying model of the chat
print("Chat model name:", chat_model.model_name)
print("Chat model details:", chat_model)

# Underlying model of the agent in the graph structure
graph = agent.get_graph()
print("Graph nodes:", list(graph.nodes.keys()))
print("Graph edges:", list(graph.edges))

# Inspect Individual Nodes
model_name_in_graph = None
for node_name, node in graph.nodes.items():
    print(f"\nNode: {node_name}")
    if hasattr(node, '__dict__'):
        pprint(vars(node))  # Print all attributes of the node
    else:
        print(f"Node object: {node}")
    if hasattr(node, 'runnable'):
        print(f"Runnable: {node.runnable}")
        if hasattr(node.runnable, 'model_name'):
            print(f"Model name: {node.runnable.model_name}")
            if node_name == 'model':
                model_name_in_graph = node.runnable.model_name

# Since the agent was created with create_agent, it should have stored config
print("Agent config:", getattr(agent, 'config', 'No config attribute'))
print("Agent attributes:", dir(agent))

# If the model name is still not showing in the Mermaid diagram, it means the underlying model
# in the agent is not directly accessible through these attributes.

Chat model name: gpt-5-nano
Chat model details: profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x7490500546e0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7490500547d0> root_client=<openai.OpenAI object at 0x749051217ce0> root_async_client=<openai.AsyncOpenAI object at 0x749050054770> model_name='gpt-5-nano' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True
Graph nodes: ['__start__', 'model', '__end__']
Graph edges: [Edge(source='__start__', target='model', data=None, conditional=False), Edge(sou

## Add web search tool

In [8]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of San Francisco?")

{'query': 'Who is the current mayor of San Francisco?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Mayor_of_San_Francisco',
   'title': 'Mayor of San Francisco',
   'content': 'The current mayor is Democrat Daniel Lurie.',
   'score': 0.95903563,
   'raw_content': None},
  {'url': 'https://apnews.com/article/san-francisco-new-mayor-liberal-city-81ea0a7b37af6cbb68aea7ef5cc6a4f0',
   'title': "San Francisco's new mayor is starting to unite the fractured ...",
   'content': 'San Francisco Mayor Daniel Lurie, a political newcomer and Levi Strauss heir, has marked his first 100 days with a hands-on, business-friendly approach.',
   'score': 0.8665964,
   'raw_content': None},
  {'url': 'https://www.sf.gov/departments--office-mayor',
   'title': 'Office of the Mayor',
   'content': 'Daniel Lurie is the 46th Mayor of the City and County of San Francisco.',
   'score': 0.8624794,
   'raw_content': None},
  {'url': 'https://

In [9]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search]
)

question = HumanMessage(content="Who is the current mayor of San Francisco?")

response = agent.invoke(
    {"messages": [question]}
)

In [10]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Who is the current mayor of San Francisco?', additional_kwargs={}, response_metadata={}, id='44a34e11-b1d4-43df-bc37-cdc10ee965c7'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 539, 'prompt_tokens': 133, 'total_tokens': 672, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CxiNagVA1I1jvBZTxlSklZzEO47FM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb9c0-d892-7e40-b6c8-a1ae58a985a3-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'current mayor of San Francisco'}, 'id': 'call_fAAihoyPZYZm2W0Ypd8EzTcN', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tok

Course example trace: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r

My Trace: https://smith.langchain.com/public/e5c37abd-577f-4e93-838f-20f9cf0edfd7/r